
<h1 id="Reading-CSV/TSV">Reading CSV/TSV<a class="anchor-link" href="#Reading-CSV/TSV">¶</a></h1>



<p>The data we'll read comes from 
<a href="https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt">https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt</a></p>



<p>We'll read data from the "Gift card" category, which is fairly small. The raw data is here, and should be downloaded to your local machine:
<a href="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz">https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz</a></p>


In [ ]:

path = "/home/lizhaoyi/datasets/amazon/amazon_reviews_us_Gift_Card_v1_00.tsv.gz"




<p>Note that the data is gzipped (filetype .gz). Rather than unzipping it, we can use the "gzip" library to read zipped data directly from the file</p>


In [ ]:

import gzip




<p>Using this library, we can open the data as if it were a regular file. "rt" converts from bytes to strings:</p>


In [ ]:

f = gzip.open(path, 'rt', encoding="utf8")




<p>Let's look at one line of the file:</p>


In [ ]:

header = f.readline()



In [ ]:

header




<p>This line is called the "header." Note that it contains the names of the fields we expect to find in the file. These fields are separeted by tabs (\t) in a tsv file.</p>
<p>We can extract these fields to a list using the "split()" function, which separates the string on the tab character:</p>


In [ ]:

header = header.strip().split('\t')



In [ ]:

header




<p>We can now do the same thing to extract every line from the file, using a "for" loop:</p>


In [ ]:

lines = []



In [ ]:

for line in f:
    fields = line.split('\t')
    lines.append(fields)




<p>Let's look at the first line:</p>


In [ ]:

lines[0]




<p>It's hard to keep track of what each field means, but note that each entry corresponds to one field from the header. Using the "zip" function, we can match the header columns to the corresponding columns of the data:</p>


In [ ]:

z = zip(header, lines[0])
list(z)




<p>Note that this data is now essentially what is known as a "Key Value" pair, where the first entry is the key, and the second is the value</p>
<p>Python has a special data structure for dealing with key value pairs known as a "dictionary". This allows us to index the data using the keys directly. Let's convert this data to a dictionary:</p>


In [ ]:

d = dict(zip(header, lines[0]))
d




<p>Now we can directly query any of the fields:</p>


In [ ]:

d['customer_id']



In [ ]:

d['star_rating']




<p>It might be useful to convert a few of the numerical fields from strings to integers:</p>


In [ ]:

d['star_rating'] = int(d['star_rating'])
d['helpful_votes'] = int(d['helpful_votes'])
d['total_votes'] = int(d['total_votes'])




<p>Finally, let's do the same thing for every line in the file, to build our dataset</p>


In [ ]:

dataset = []



In [ ]:

for line in lines:
    # Convert to key-value pairs
    d = dict(zip(header, line))
    # Convert strings to integers for some fields:
    d['star_rating'] = int(d['star_rating'])
    d['helpful_votes'] = int(d['helpful_votes'])
    d['total_votes'] = int(d['total_votes'])
    dataset.append(d)




<p>Now, we can easily perform queries on any entry in our dataset:</p>


In [ ]:

dataset[50]['star_rating']




<p>Finally, while we've done these operations manually above, the same can be accomplished using the python csv library. Doing so saves us a few lines:</p>


In [ ]:

import csv



In [ ]:

c = csv.reader(gzip.open(path, 'rt'), delimiter = '\t')
dataset = []



In [ ]:

first = True
for line in c:
    # The first line is the header
    if first:
        header = line
        first = False
    else:
        d = dict(zip(header, line))
        # Convert strings to integers for some fields:
        d['star_rating'] = int(d['star_rating'])
        d['helpful_votes'] = int(d['helpful_votes'])
        d['total_votes'] = int(d['total_votes'])
        dataset.append(d)



In [ ]:

dataset[20]




<h3 id="Avoiding-reading-large-files-into-disk">Avoiding reading large files into disk<a class="anchor-link" href="#Avoiding-reading-large-files-into-disk">¶</a></h3>



<p>Note that it can be rather costly (in terms of memory) to read the entire file into a data structure, when we may only need to manipulate a small part of it at any one time. So, rather than reading the entire dataset into a data structure, this time we'll want to perform pre-processing on the dataset as we read each line.</p>
<p>Let's suppose that for this exercise, we only care about extracting user, items, ratings, timestamps, and the "verified_purchase" flag:</p>


In [ ]:

dataset = []
header = f.readline().strip().split('\t')
for line in f:
    line = line.split('\t')
    d = dict(zip(header, line))
    d['star_rating'] = int(d['star_rating'])
    d2 = {}
    for field in ['star_rating', 'customer_id', 'product_id', 'review_date', 'verified_purchase']:
        d2[field] = d[field]
    dataset.append(d2)




<h1 id="Computing-simple-statistics-from-data">Computing simple statistics from data<a class="anchor-link" href="#Computing-simple-statistics-from-data">¶</a></h1>



<p>Let's quickly read the data again using the same code:</p>


In [ ]:

import gzip
path = "/home/lizhaoyi/datasets/amazon/amazon_reviews_us_Gift_Card_v1_00.tsv.gz"
f = gzip.open(path, 'rt')



In [ ]:

dataset = []
# Read the header:
header = f.readline().strip().split('\t')
for line in f:
    # Separate by tabs
    line = line.split('\t')
    # Convert to key-value pairs
    d = dict(zip(header, line))
    # Convert strings to integers for some fields:
    d['star_rating'] = int(d['star_rating'])
    d['helpful_votes'] = int(d['helpful_votes'])
    d['total_votes'] = int(d['total_votes'])
    dataset.append(d)




<p>By iterating through our dataset, we can straightforwardly compute some simple statistics, e.g. how many ratings are there?</p>


In [ ]:

nRatings = len(dataset)
nRatings




<p>And what is the average rating?</p>


In [ ]:

average = 0
for d in dataset:
    average += d['star_rating']
average /= nRatings
average




<p>How many unique users and products are there in this dataset?</p>


In [ ]:

users = set()
items = set()
for d in dataset:
    users.add(d['customer_id'])
    items.add(d['product_id'])

len(users),len(items)




<h3 id="E.g.-What-is-the-average-rating-of-a-verified-purchase,-versus-an-unverified-purchase?">E.g. What is the average rating of a verified purchase, versus an unverified purchase?<a class="anchor-link" href="#E.g.-What-is-the-average-rating-of-a-verified-purchase,-versus-an-unverified-purchase?">¶</a></h3>


In [ ]:

avVerified = 0
avUnverified = 0
nVerified = 0
nUnverified = 0
for d in dataset:
    if d['verified_purchase'] == 'Y':
        avVerified += d['star_rating']
        nVerified += 1
    else:
        avUnverified += d['star_rating']
        nUnverified += 1

avVerified /= nVerified
avUnverified /= nUnverified
avVerified, avUnverified




<p>Many of these types of operations can be done more easily using operations known as "list comprehensions", which allow us to process and filter the data:</p>


In [ ]:

verifiedRatings = [d['star_rating'] for d in dataset if d['verified_purchase'] == 'Y']
unverifiedRatings = [d['star_rating'] for d in dataset if d['verified_purchase'] == 'N']



In [ ]:

print(sum(verifiedRatings) * 1.0 / len(verifiedRatings))
print(sum(unverifiedRatings) * 1.0 / len(unverifiedRatings))




<h1 id="Reading-data-from-JSON">Reading data from JSON<a class="anchor-link" href="#Reading-data-from-JSON">¶</a></h1>



<p>Another common data format is JSON (<a href="https://www.json.org/">https://www.json.org/</a>). This format generalizes key-value pairs (like those that we saw in the previous notebooks), by allowing the values to <em>also</em> be key value pairs (allowing for hierarchical data).</p>



<p>Let's look at an example of such data, this time from Yelp. This data is part of the "Yelp dataset challenge" and should first be downloaded locally before beginning this notebook:
<a href="https://www.yelp.com/dataset/download">https://www.yelp.com/dataset/download</a></p>


In [ ]:

path = "/home/lizhaoyi/datasets/yelp_data/review.json"




<p>This file is very large -- for the moment let's just look at the first 50,000 lines</p>


In [ ]:

f = open(path, 'r')



In [ ]:

lines = []
for i in range(50000):
    lines.append(f.readline())




<p>Let's just look at the first line:</p>


In [ ]:

lines[0]




<p>Note that this looks very much like a python dictionary! In fact we could convert it directly to a python dictionary using the "eval" operator:</p>


In [ ]:

d = eval(lines[0])
d




<p>Then we could treat it just like a key-value pair:</p>


In [ ]:

d['user_id']



In [ ]:

d['stars']




<p>The "eval" operator isn't the safest though -- it's basically executing the line of the file as if it were native python code. This is a dangerous thing to do, especially if we don't trust the source of the file we're using.</p>
<p>More safely, we can use the json library to read the data.</p>


In [ ]:

import json




<p>and then read the data in the same way:</p>


In [ ]:

d = json.loads(lines[0])
d




<p>Let's look at a different dataset, also from the yelp challenge. This time let's read the business metadata:</p>


In [ ]:

path = "/home/lizhaoyi/datasets/yelp_data/business.json"
f = open(path, 'r')



In [ ]:

dataset = []
for i in range(50000):
    dataset.append(json.loads(f.readline()))



In [ ]:

dataset[0]




<p>Again, each entry is a set of key-value, pairs, but note that some of the values are <em>themselves</em> key value pairs:</p>


In [ ]:

dataset[0]['attributes']




<p>Note also that (at least in this dataset) numerical values are already formatted as ints/floats, and don't need to be converted:</p>


In [ ]:

dataset[0]['stars']




<h1 id="Time-and-date-data">Time and date data<a class="anchor-link" href="#Time-and-date-data">¶</a></h1>


In [ ]:

import json
path = "/home/lizhaoyi/datasets/yelp_data/review.json"
f = open(path, 'r')



In [ ]:

dataset = []
for i in range(50000):
    dataset.append(json.loads(f.readline()))



In [ ]:

dataset[0]




<p>Let's look at the first review's date:</p>


In [ ]:

timeString = dataset[0]['date']
print(timeString)




<p>To handle the string-formatted time data, we can use python's "time" library:</p>


In [ ]:

import time



In [ ]:

timeStruct = time.strptime(timeString, "%Y-%m-%d")
timeStruct




<p>The above operation produced a data structure repreresting the time string. Note that we had to specify a format, here a four digit year, two digit month and day, separated by hyphens. Details about formatting strings can be found on <a href="https://docs.python.org/2/library/datetime.html">https://docs.python.org/2/library/datetime.html</a>. E.g. we could also have a string consisting of a time and date:</p>


In [ ]:

time.strptime("21:36:18, 28/5/2019", "%H:%M:%S, %d/%m/%Y")




<p>The above time structure allows us to extract specific information about the date/timestamp, but easier yet might be to convert the timestamps into integers for easier comparison:</p>


In [ ]:

timeInt = time.mktime(timeStruct)
timeInt




<p>The exact value of this time is the <em>number of seconds since 1970</em>. While not the most obvious format, it does allow for easy comparison between times:</p>


In [ ]:

timeInt2 = time.mktime(time.strptime(dataset[99]['date'], "%Y-%m-%d"))




<p>E.g. the number of seconds between dataset[0] and dataset[99] is...</p>


In [ ]:

timeDiff = timeInt - timeInt2
timeDiff



In [ ]:

timeDiff / 60 # minutes



In [ ]:

timeDiff / (60*60) # hours



In [ ]:

timeDiff / (60*60*24) # days




<p>The values can also be converted back to a structured time object:</p>


In [ ]:

time.gmtime(timeInt)




<p>Or we can determine, for example, what the date would be one week later:</p>


In [ ]:

time.gmtime(timeInt + 60*60*24*7)




<p>Finally It might be useful to augment our dataset to include these numerical time measurements for every sample:</p>


In [ ]:

datasetWithTimeValues = []



In [ ]:

for d in dataset:
    d['date']
    d['timeStruct'] = time.strptime(d['date'], "%Y-%m-%d")
    d['timeInt'] = time.mktime(d['timeStruct'])
    datasetWithTimeValues.append(d)




<p>The same strategy can also be used to deal with times that have no date attached to them, e.g. let's look at the length of a business's opening hours:</p>


In [ ]:

path = "/home/lizhaoyi/datasets/yelp_data/business.json"
f = open(path, 'r')
dataset = []
for i in range(50000):
    dataset.append(json.loads(f.readline()))



In [ ]:

dataset[0]['hours']




<p>Let's try to calculate how long the business is open on Fridays:</p>


In [ ]:

hours = dataset[0]['hours']['Friday']
hours



In [ ]:

openTime,closeTime = hours.split('-')
openTime,closeTime



In [ ]:

timeIntOpen = time.mktime(time.strptime(openTime, "%H:%M"))
timeIntClose = time.mktime(time.strptime(closeTime, "%H:%M"))



In [ ]:

timeIntOpen



In [ ]:

timeIntClose




<p>Note that since we specified no date, these timestamps are assumed to correspond to January 1, 1900:</p>


In [ ]:

time.gmtime(timeIntOpen)




<p>However for the purposes of measuring the opening duration, it's sufficient to compute the difference:</p>


In [ ]:

(timeIntClose - timeIntOpen) / (60*60)




<h1 id="Using-pandas-for-Data-Analysis">Using pandas for Data Analysis<a class="anchor-link" href="#Using-pandas-for-Data-Analysis">¶</a></h1>



<p><strong>Case Study: Amazon Dataset</strong></p>
<p>We'll again use the the "Gift card" dataset.  Please unzip the file after downloading it and place <strong>amazon_reviews_us_Gift_Card_v1_00.tsv</strong> under the datasets.</p>
<p><strong>Objective:</strong> We seek to draw inferences from this dataset, whilst exploring some of the functionalities pandas has to offer.</p>


In [ ]:

import pandas as pd
giftcard = pd.read_csv('./datasets/amazon_reviews_us_Gift_Card_v1_00.tsv', sep='\t')
print(type(giftcard))
giftcard.head()



In [ ]:

giftcard.describe()



In [ ]:

giftcard['star_rating'].max()



In [ ]:

giftcard['star_rating'].min()



In [ ]:

giftcard['star_rating'].mean()



In [ ]:

giftcard.isnull().any()



In [ ]:

giftcard.shape



In [ ]:

giftcard = giftcard.dropna()



In [ ]:

giftcard.shape



In [ ]:

df = giftcard[['star_rating','helpful_votes','total_votes']]
df



In [ ]:

df.head(100).plot.bar()



In [ ]:

df.head(100).plot.hist()



In [ ]:

df.head(100).plot()



In [ ]:

df_helpful_votes = df[df['helpful_votes'] > 0]
print(df.shape)
print(df_helpful_votes.shape)



In [ ]:

df_helpful_votes
df_helpful_votes.groupby('total_votes').mean()



In [ ]:

df_helpful_votes[['helpful_votes','total_votes']].head(50).plot.bar()



In [ ]:

%matplotlib inline
giftcard.hist(column='star_rating', figsize=(15,10))




<h1 id="Plotting-with-matplotlib">Plotting with matplotlib<a class="anchor-link" href="#Plotting-with-matplotlib">¶</a></h1>



<p>Let's do a simple plot of ratings as a function of the day of the week.</p>



<p>First we'll import the <em>matplotlib</em> library:</p>


In [ ]:

from matplotlib import pyplot as plt
from collections import defaultdict



In [ ]:

weekRatings = defaultdict(list)



In [ ]:

for d in datasetWithTimeValues:
    day = d['timeStruct'].tm_wday
    weekRatings[day].append(d['stars'])



In [ ]:

weekAverages = {}



In [ ]:

for d in weekRatings:
    weekAverages[d] = sum(weekRatings[d]) * 1.0 / len(weekRatings[d])



In [ ]:

weekAverages



In [ ]:

X = list(weekAverages.keys())



In [ ]:

Y = [weekAverages[x] for x in X]



In [ ]:

plt.plot(X, Y)




<p>It might be nicer to plot this as a bar plot:</p>


In [ ]:

plt.bar(X, Y)




<p>Let's zoom in to make the differences a bit more visible:</p>


In [ ]:

plt.ylim(3.6, 3.8)
plt.bar(X, Y)




<p>Next let's add some labels:</p>


In [ ]:

plt.ylim(3.6, 3.8)
plt.xlabel("Weekday")
plt.ylabel("Rating")
plt.title("Rating as a function of weekday")
plt.bar(X, Y)




<p>Finally let's rename the ticks to correspond to the days of the week</p>


In [ ]:

plt.ylim(3.6, 3.8)
plt.xlabel("Weekday")
plt.ylabel("Rating")
plt.xticks([0,1,2,3,4,5,6],['S', 'M', 'T', 'W', 'T', 'F', 'S'])
plt.title("Rating as a function of weekday")
plt.bar(X, Y)

